# Analysis of Detachment Study

In [18]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import collections
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.colours as col
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [2]:
import importlib
importlib.reload(mgut)

<module 'flopter.magnum.utils' from '/home/jleland/coding/projects/flopter/flopter/magnum/utils.py'>

In [3]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'analysed_3_downsampled'
# path_to_analysed_datasets = 'analysed_4'
path_to_analysed_datasets = 'analysed_4_downsampled'
# path_to_analysed_datasets = 'analysed_5_downsampled'
os.chdir(path_to_datasets)

In [4]:
magnum_probes = lp.MagnumProbes()

## Load adc file metadata

In [5]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
# print(meta_data_ds)

In [6]:
tcp_ds = xr.open_dataarray('target_pressure.nc')
tcp_ds

<xarray.DataArray 'shot_target_pressure' (shot_number: 523)>
array([0.149541, 0.152593, 0.143437, ..., 2.25837 , 2.639851, 3.054903])
Coordinates:
  * shot_number  (shot_number) int64 0 1 2 3 4 5 6 ... 517 518 519 520 521 522

In [7]:
meta_data_ds['shot_target_pressure'] = tcp_ds
meta_data_ds

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int64 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_probe             (shot_number) object ...
    adc_5_probe             (shot_number) object ...
    ts_temp_max             (shot_number) float64 ...
    ts_dens_max             (shot_number) float64 ...
    shot_end_time           (shot_number) datetime64[ns] ...
    ts_fl                   (shot_number) bool ...
    shot_time               (shot_number) datetime64[ns] ...
    shot_source_current     (shot_number) float64 ...
    shot_b_field            (shot_number) float64 ...
    shot_tilt               (shot_number) float64 ...
    shot_target_pos         (shot_number) float64 ...
    shot_target_rot         (shot_number) float64 ...
    shot_target_pumpspeed   (shot_number) int32 ...
    shot_source_pumpspeed   (shot_number) int32 ...
    shot_heating_pumpspeed  (shot_number) int32 ...
    shot_source_pos         (shot_number) float64 ...
    shot_deuterium_gf       (shot_number) float64 ...
    shot_hydrogen_gf        (shot_number) float64 ...
    shot_helium_gf          (shot_number) float64 ...
    shot_hydrogen_puffed    (shot_number) float64 ...
    shot_helium_puffed      (shot_number) float64 ...
    adc_timestamp           (shot_number) float64 ...
    adc_4_coax              (shot_number) float64 ...
    adc_4_shunt_resistance  (shot_number) float64 ...
    adc_5_coax              (shot_number) float64 ...
    adc_5_shunt_resistance  (shot_number) float64 ...
    adc_calibration_fl      (shot_number) float64 ...
    shot_fl                 (shot_number) float64 ...
    acquisition_length      (shot_number) float64 ...
    adc_end_time            (shot_number) datetime64[ns] ...
    adc_end_timestamp       (shot_number) float64 ...
    adc_freqs               (shot_number) float64 ...
    shot_target_pressure    (shot_number) float64 0.1495 0.1526 ... 2.64 3.055

## Indices for Anglescan

In [8]:
detachment_indices = [515,516,517,518,519,520,521,522,523,524,525,526]

In [9]:
droppable_vars = {
    'adc_4_probe',
    'adc_5_probe',
    'adc_calibration_index',
    'adc_end_time',
    'adc_filename',
    'adc_folder',
    'adc_index',
    'adc_time',
    'shot_end_time',
    'sweep_time',
    'ts_number',
    'ts_time',
    'ts_timestamp'
}

In [10]:
detachment_ds = mgut.get_dataset_from_indices(detachment_indices, anglescan_fl=False, average_direction_fl=False,
                                              path_to_analysed_datasets=path_to_analysed_datasets, 
#                                               check_sweep_trim_fl=True,
                                              ).drop(droppable_vars)

In [11]:
detachment_ds

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 12, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'S' 'L'
  * shot_number             (shot_number) int32 511 512 513 514 ... 520 521 522
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 8.0 8.0 8.0 ... 8.0 8.0 8.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 -86.96 ... -4.888
    current                 (shot_number, probe, time) float64 -0.3533 ... 0.04652
    shot_time               (shot_number, probe, time) float64 4.604 ... 4.493
    start_time              (shot_number, probe) float64 4.601 4.601 ... 4.49
    d_current               (shot_number, probe, time) float64 0.00108 ... 0.0005395
    std_current             (shot_number, probe, time) float64 0.03905 ... 0.01973
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    ts_temp_max             (shot_number) float64 ...
    ts_dens_max             (shot_number) float64 ...
    ts_fl                   (shot_number) bool ...
    shot_source_current     (shot_number) float64 ...
    shot_b_field            (shot_number) float64 ...
    shot_tilt               (shot_number) float64 7.98 7.98 7.98 ... 7.98 7.98
    shot_target_pos         (shot_number) float64 ...
    shot_target_rot         (shot_number) float64 ...
    shot_target_pumpspeed   (shot_number) int32 ...
    shot_source_pumpspeed   (shot_number) int32 ...
    shot_heating_pumpspeed  (shot_number) int32 ...
    shot_source_pos         (shot_number) float64 ...
    shot_deuterium_gf       (shot_number) float64 ...
    shot_hydrogen_gf        (shot_number) float64 ...
    shot_helium_gf          (shot_number) float64 ...
    shot_hydrogen_puffed    (shot_number) float64 ...
    shot_helium_puffed      (shot_number) float64 ...
    adc_timestamp           (shot_number) float64 ...
    adc_4_coax              (shot_number) float64 ...
    adc_4_shunt_resistance  (shot_number) float64 ...
    adc_5_coax              (shot_number) float64 ...
    adc_5_shunt_resistance  (shot_number) float64 ...
    adc_calibration_fl      (shot_number) float64 ...
    shot_fl                 (shot_number) float64 ...
    acquisition_length      (shot_number) float64 ...
    adc_end_timestamp       (shot_number) float64 ...
    adc_freqs               (shot_number) float64 ...

In [12]:
mgut.plot_densscan_ivs(detachment_ds)

(<Figure size 640x480 with 2 Axes>,
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7fd6da670a58>,
       dtype=object))

In [13]:
fit_data_df = mgut.fit_magnum_ds(detachment_ds, probes=('L', 'S'), threshold=None, scan_param='shot_number', plot_fl=False, mode=0, trimming_vals=(0.3, 0.1, 0.02))

In [14]:
fit_ds = mgut.combine_fit_ds(fit_data_df, detachment_ds, probes=['L', 'S'], index_dim='shot_number')

In [15]:
fit_ds['shot_target_pressure'] = tcp_ds
fit_ds

<xarray.Dataset>
Dimensions:                 (probe: 2, shot_number: 12, time: 50, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int64 511 512 513 514 ... 520 521 522
  * probe                   (probe) object 'L' 'S'
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 8.0 8.0 8.0 ... 8.0 8.0 8.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, probe, time) float64 -88.9 ... -6.38
    current                 (shot_number, probe, time) float64 -0.1783 ... 0.1813
    shot_time               (shot_number, probe, time) float64 4.604 ... 4.493
    start_time              (shot_number, probe) float64 4.601 4.601 ... 4.49
    d_current               (shot_number, probe, time) float64 0.001755 ... 0.0007504
    std_current             (shot_number, probe, time) float64 0.06346 ... 0.02745
    ts_density              (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_temperature        (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temp_max             (shot_number) float64 6.159 7.567 ... 1.365 1.151
    ts_dens_max             (shot_number) float64 9.285e+19 ... 3.083e+20
    ts_fl                   (shot_number) bool True True True ... True True True
    shot_source_current     (shot_number) float64 135.4 135.5 ... 135.3 135.4
    shot_b_field            (shot_number) float64 1.5 1.5 1.5 ... 1.5 1.5 1.5
    shot_tilt               (shot_number) float64 7.98 7.98 7.98 ... 7.98 7.98
    shot_target_pos         (shot_number) float64 -0.127 -0.127 ... -0.127
    shot_target_rot         (shot_number) float64 -89.94 -89.94 ... -89.94
    shot_target_pumpspeed   (shot_number) int32 82 60 40 25 25 ... 25 25 25 25
    shot_source_pumpspeed   (shot_number) int32 82 82 82 82 82 ... 82 82 82 82
    shot_heating_pumpspeed  (shot_number) int32 82 82 82 82 82 ... 82 82 82 82
    shot_source_pos         (shot_number) float64 1.2 1.2 1.2 ... 1.2 1.2 1.2
    shot_deuterium_gf       (shot_number) float64 0.004722 0.008212 ... 0.01084
    shot_hydrogen_gf        (shot_number) float64 9.333 9.329 ... 9.342 9.322
    shot_helium_gf          (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_hydrogen_puffed    (shot_number) float64 -0.0151 -0.01705 ... 6.755
    shot_helium_puffed      (shot_number) float64 -0.05492 -0.0477 ... -0.05601
    adc_timestamp           (shot_number) float64 6.7e+18 6.7e+18 ... 6.7e+18
    adc_4_coax              (shot_number) float64 3.0 3.0 3.0 ... 3.0 3.0 3.0
    adc_4_shunt_resistance  (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_5_coax              (shot_number) float64 4.0 4.0 4.0 ... 4.0 4.0 4.0
    adc_5_shunt_resistance  (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_calibration_fl      (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_fl                 (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    acquisition_length      (shot_number) float64 10.0 10.0 10.0 ... 10.0 10.0
    adc_end_timestamp       (shot_number) float64 6.7e+18 6.7e+18 ... 6.7e+18
    adc_freqs               (shot_number) float64 1e+06 1e+06 ... 1e+06 1e+06
    B                       (probe, shot_number) float64 1.5 1.5 1.5 ... 1.5 1.5
    ts_temp                 (probe, shot_number) object 6.158978753446999 ... 1.150592237748636
    ts_dens                 (probe, shot_number) object 9.285065194484636e+19 ... 3.0833845590671176e+20
    fit_success_fl          (probe, shot_number) object True True ... True True
    temp                    (probe, shot_number) float64 1.461 1.588 ... 1.104
    d_temp                  (probe, shot_number) float64 0.03618 ... 0.05042
    isat                    (probe, shot_number) float64 0.1462 ... 0.1405
    d_isat                  (pr

## Plotting

In [19]:
# The default colours from pyplot
# tableau_palette = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',]
# markers = ['.', '+', 'x', '^', '*', 'o']
# probe_colours = {
#     'L': 'blue',
#     'S': 'red',
#     'B': 'green',
#     'R': 'gold'
# }
probe_colours = {
    'L': col.palettes['c'][3],
    'S': col.palettes['c'][2],
    'B': col.palettes['c'][1],
    'R': col.palettes['b'][2],
#     'TS': col.palettes['d'][-1],
}
tableau_palette = ['tab:blue', 'tab:green', 'tab:red', 'black', 'tab:purple', 'tab:grey', 'tab:orange']
markers = ['.', '+', 'x', '^', '*', 'o', 's']
global_options = {'fontsize':12, }

probes = ['L', 'S', 'B', 'R']
# probe_colours = {probes[i]: tableau_palette[i] for i in range(len(probes))}


In [64]:
fig, ax = plt.subplots(1, 2, sharex=True, figsize=[10,5])
# fit_ds.sel(shot_number=511)['ts_temperature'].plot.line(hue='shot_number', ax=ax[0])
# fit_ds.sel(shot_number=511)['ts_density'].plot.line(hue='shot_number', ax=ax[1])

ts_ds = fit_ds.sel(shot_number=511)

ax[0].errorbar('ts_radial_pos', 'ts_temperature', yerr='ts_d_temperature', data=ts_ds, color=col.palettes['d'][-1], 
               fmt='+', label='')
ax[1].errorbar('ts_radial_pos', 'ts_density', yerr='ts_d_density', data=ts_ds, color=col.palettes['d'][-1], fmt='+', 
               label='')

# colours = {
#     'L': 'blue',
#     'S': 'red',
#     'B': 'green',
#     'R': 'gold'
# }
probe_linestyles = {
    'L': '-',
    'S': '--',
    'B': '-.',
    'R': (0, (1.0, 1.0)),
}
colours = probe_colours
for probe, pos in magnum_probes.probe_position.items():
    probe = probe.upper()
    pos -= 1.75
    ax[0].axvline(x=pos, linestyle='dashed', linewidth=1.5, color=colours[probe], label=probe, ls=probe_linestyles[probe])
    ax[1].axvline(x=pos, linestyle='dashed', linewidth=1.5, color=colours[probe], label=probe, ls=probe_linestyles[probe])

# ax[1].legend()
ax[1].legend(title='Probe Position')
ax[0].set_ylabel(r'Electron Temperature [eV]')
ax[1].set_ylabel(r'Electron Density [m$^{-3}$]')
ax[0].set_xlabel(r'TS Radial Position [mm]')
ax[1].set_xlabel(r'TS Radial Position [mm]')

fig.tight_layout()


In [47]:
ts_probe_denss['L'].shape

(12, 3)

In [65]:
fig, ax = plt.subplots(2, 2, sharex=True, figsize=[8,6])

ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(fit_ds, aggregate_dims=None, offset=-1.75)

markers = [
    'o',
    '^'
]

for i, probe in enumerate(fit_ds.probe.values):
    ds = fit_ds.sel(probe=probe) #.sortby('shot_target_pos')
    ax[0,0].errorbar('shot_target_pressure', 'temp', yerr='d_temp', data=ds, label=probe, linestyle='none', marker=markers[i],
                     color=probe_colours[probe], mfc='none')
#     ax[0,0].plot('shot_target_pressure', 'ts_max_temp', data=ds, label=probe, linestyle='none', marker='.')
    
    
    A_coll = magnum_probes[probe].get_collection_area(np.radians(8.0))
#     s_q_par = (s_shots_scan['isat'].values / magnum_probes['S'].get_collection_area(np.radians(scan_tilts[i]))) * 7 * s_shots_scan['temp'].values
    J_sat = ds['isat'].values / A_coll
    q_par = J_sat * 7 * ds['temp'].values
    ts_q_par = ((lp.sound_speed(ts_probe_temps[probe][:, 0]) 
                 * ts_probe_denss[probe][:, 0] * 1.6e-19) * 7 * ts_probe_temps[probe][:, 0])
    
    ax[1,0].errorbar('shot_target_pressure', 'dens', yerr='d_dens', data=ds, label=probe, linestyle='none', marker=markers[i],
                     color=probe_colours[probe], mfc='none')
    ax[0,1].errorbar(ds['shot_target_pressure'], J_sat, label=probe, linestyle='none', marker=markers[i],
                     color=probe_colours[probe], mfc='none')
    ax[1,1].errorbar(ds['shot_target_pressure'], q_par, fmt=markers[i], label=probe, color=probe_colours[probe], mfc='none')
    ax[1,1].errorbar(ds['shot_target_pressure'], ts_q_par, fmt=markers[i], label=f'TS at {probe}', color='grey', mfc='none')
    
    
    ax[0,0].plot(ds['shot_target_pressure'], ts_probe_temps[probe][:, 0], 'x', label='TS', color='grey')
    ax[1,0].plot(ds['shot_target_pressure'], ts_probe_denss[probe][:, 0], 'x', label='TS', color='grey')
    
    
ax[0,0].legend()
ax[1,0].legend()
# ax[0,0].axhline(y=fit_ds['ts_temp'].values[0][0], **c.AX_LINE_DEFAULTS)
# ax[1,0].axhline(y=fit_ds['ts_dens'].values[0][0], **c.AX_LINE_DEFAULTS)
ax[1,1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[1,1].set_yscale('log')

ax[0,0].set_ylabel(r'$T_e \; [eV]$')
ax[1,0].set_ylabel(r'$n_e \; [m^{-3}]$')
ax[1,0].set_yscale(r'log')
ax[0,1].set_ylabel(r'$I_{sat}$ [A]')
ax[1,1].set_ylabel(r'$\chi^2_{\nu}$')

ax[1,0].set_xlabel('Target Chamber Pressure [?]')
ax[1,1].set_xlabel('Target Chamber Pressure [?]')

fig.tight_layout()

l 4.25
s -5.75
b -15.75
r -25.75


In [68]:
fig, ax = plt.subplots(2, 2, sharex=True, figsize=[8,6])

ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(fit_ds, aggregate_dims=None) #, offset=-1.75)

markers = [
    'o',
    '^'
]

for i, probe in enumerate(fit_ds.probe.values):
    ds = fit_ds.sel(probe=probe) #.sortby('shot_target_pos')
    ax[0,0].errorbar('shot_target_pressure', 'temp', yerr='d_temp', data=ds, label=probe, linestyle='none', marker=markers[i],
                     color=probe_colours[probe], mfc='none')
#     ax[0,0].plot('shot_target_pressure', 'ts_max_temp', data=ds, label=probe, linestyle='none', marker='.')
    
    
    A_coll = magnum_probes[probe].get_collection_area(np.radians(8.0))
#     s_q_par = (s_shots_scan['isat'].values / magnum_probes['S'].get_collection_area(np.radians(scan_tilts[i]))) * 7 * s_shots_scan['temp'].values
    J_sat = ds['isat'].values / A_coll
    q_par = J_sat * 7 * ds['temp'].values
    ts_q_par = ((lp.sound_speed(ts_probe_temps[probe][:, 0]) 
                 * ts_probe_denss[probe][:, 0] * 1.6e-19) * 7 * ts_probe_temps[probe][:, 0])
    
    ax[1,0].errorbar('shot_target_pressure', 'dens', yerr='d_dens', data=ds, label=probe, linestyle='none', marker=markers[i],
                     color=probe_colours[probe], mfc='none')
    ax[0,1].errorbar(ds['shot_target_pressure'], J_sat, label=probe, linestyle='none', marker=markers[i],
                     color=probe_colours[probe], mfc='none')
    ax[1,1].errorbar(ds['shot_target_pressure'], q_par, fmt=markers[i], label=probe, color=probe_colours[probe], mfc='none')
    ax[1,1].errorbar(ds['shot_target_pressure'], ts_q_par, fmt=markers[i], label=f'TS at {probe}', color='grey', mfc='none')
    
    
    ax[0,0].plot(ds['shot_target_pressure'], ts_probe_temps[probe][:, 0], markers[i], 
                 label=f'TS at {probe}', color='grey', mfc='none')
    ax[1,0].plot(ds['shot_target_pressure'], ts_probe_denss[probe][:, 0], markers[i], 
                 label='TS', color='grey', mfc='none')
    
    
ax[0,0].legend()
ax[0,0].set_xscale('log')
# ax[1,0].legend()
# ax[0,0].axhline(y=fit_ds['ts_temp'].values[0][0], **c.AX_LINE_DEFAULTS)
# ax[1,0].axhline(y=fit_ds['ts_dens'].values[0][0], **c.AX_LINE_DEFAULTS)
# ax[1,1].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
ax[1,1].set_yscale('log')

ax[0,0].set_ylabel(r'$T_e \; [eV]$')
ax[1,0].set_ylabel(r'$n_e \; [m^{-3}]$')
ax[1,0].set_yscale(r'log')
ax[0,1].set_ylabel(r'$J_{sat}$ [Am$^{-2}$]')
ax[1,1].set_ylabel(r'$q_\parallel$ [Wm$^{-2}$]')

ax[1,0].set_xlabel('Target Chamber Pressure [mTorr]')
ax[1,1].set_xlabel('Target Chamber Pressure [mTorr]')

fig.tight_layout()

l 6
s -4
b -14
r -24


In [74]:
fig, ax = plt.subplots()

ts_probe_temps, ts_probe_denss, ts_probe_d_temps, ts_probe_d_denss = mgut.interpolate_ts_position(fit_ds, aggregate_dims=None) #, offset=-1.75)

markers = [
    'o',
    '^'
]

for i, probe in enumerate(fit_ds.probe.values):
    ds = fit_ds.sel(probe=probe) #.sortby('shot_target_pos')
    
    lambda_D = lp.debye_length(ds['temp'].values, ds['dens'].values)
    larmor_r = lp.ion_larmor_radius(ds['temp'].values, ds.B)
    
    ts_lambda_D = lp.debye_length(ts_probe_temps[probe][:, 0], ts_probe_denss[probe][:, 0])
    ts_larmor_r = lp.ion_larmor_radius(ts_probe_temps[probe][:, 0], ds.B)
    
    ax.errorbar(ds['shot_target_pressure'], larmor_r / lambda_D, label=probe, linestyle='none', marker=markers[i],
                     color=probe_colours[probe], mfc='none')
    
    ax.plot(ds['shot_target_pressure'], ts_larmor_r / ts_lambda_D, markers[i], 
                 label=f'TS at {probe}', color='grey', mfc='none')
#     ax[1,0].plot(ds['shot_target_pressure'], ts_probe_denss[probe][:, 0], markers[i], 
#                  label='TS', color='grey', mfc='none')
    
    
    
ax.legend()
ax.set_xscale('log')
ax.set_ylabel(r'$\xi \;$ [unitless]')
ax.set_yscale(r'log')

ax.set_xlabel('Target Chamber Pressure [mTorr]')

fig.tight_layout()

l 6
s -4
b -14
r -24


Need to get target chamber pressure out of the magnum database

In [27]:
ds

<xarray.Dataset>
Dimensions:                 (shot_number: 12, time: 50, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int64 511 512 513 514 ... 520 521 522
    probe                   <U1 'S'
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
    tilt                    (shot_number) float64 8.0 8.0 8.0 ... 8.0 8.0 8.0
Dimensions without coordinates: time
Data variables:
    voltage                 (shot_number, time) float64 -86.89 -85.51 ... -6.401
    current                 (shot_number, time) float64 -0.3583 ... 0.1824
    shot_time               (shot_number, time) float64 4.46 4.46 ... 4.371
    start_time              (shot_number) float64 4.458 4.477 ... 4.454 4.369
    d_current               (shot_number, time) float64 0.001271 ... 0.0007505
    std_current             (shot_number, time) float64 0.04717 ... 0.02751
    ts_density              (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_d_temperature        (shot_number, ts_radial_pos) float64 nan nan ... nan
    ts_temp_max             (shot_number) float64 6.159 7.567 ... 1.365 1.151
    ts_dens_max             (shot_number) float64 9.285e+19 ... 3.083e+20
    ts_fl                   (shot_number) bool True True True ... True True True
    shot_source_current     (shot_number) float64 135.4 135.5 ... 135.3 135.4
    shot_b_field            (shot_number) float64 1.5 1.5 1.5 ... 1.5 1.5 1.5
    shot_tilt               (shot_number) float64 7.98 7.98 7.98 ... 7.98 7.98
    shot_target_pos         (shot_number) float64 -0.127 -0.127 ... -0.127
    shot_target_rot         (shot_number) float64 -89.94 -89.94 ... -89.94
    shot_target_pumpspeed   (shot_number) int32 82 60 40 25 25 ... 25 25 25 25
    shot_source_pumpspeed   (shot_number) int32 82 82 82 82 82 ... 82 82 82 82
    shot_heating_pumpspeed  (shot_number) int32 82 82 82 82 82 ... 82 82 82 82
    shot_source_pos         (shot_number) float64 1.2 1.2 1.2 ... 1.2 1.2 1.2
    shot_deuterium_gf       (shot_number) float64 0.004722 0.008212 ... 0.01084
    shot_hydrogen_gf        (shot_number) float64 9.333 9.329 ... 9.342 9.322
    shot_helium_gf          (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_hydrogen_puffed    (shot_number) float64 -0.0151 -0.01705 ... 6.755
    shot_helium_puffed      (shot_number) float64 -0.05492 -0.0477 ... -0.05601
    adc_timestamp           (shot_number) float64 6.7e+18 6.7e+18 ... 6.7e+18
    adc_4_coax              (shot_number) float64 3.0 3.0 3.0 ... 3.0 3.0 3.0
    adc_4_shunt_resistance  (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_5_coax              (shot_number) float64 4.0 4.0 4.0 ... 4.0 4.0 4.0
    adc_5_shunt_resistance  (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_calibration_fl      (shot_number) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_fl                 (shot_number) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    acquisition_length      (shot_number) float64 10.0 10.0 10.0 ... 10.0 10.0
    adc_end_timestamp       (shot_number) float64 6.7e+18 6.7e+18 ... 6.7e+18
    adc_freqs               (shot_number) float64 1e+06 1e+06 ... 1e+06 1e+06
    B                       (shot_number) float64 1.5 1.5 1.5 ... 1.5 1.5 1.5
    ts_temp                 (shot_number) object 6.158978753446999 ... 1.150592237748636
    ts_dens                 (shot_number) object 9.285065194484636e+19 ... 3.0833845590671176e+20
    fit_success_fl          (shot_number) object True True True ... True True
    temp                    (shot_number) float64 2.498 2.3 ... 1.106 1.093
    d_temp                  (shot_number) float64 0.06617 0.0455 ... 0.04911
    isat                    (shot_number) float64 0.2671 0.2689 ... 0.1413
    d_isat                  (shot_number) float64 0.0007747 ... 0.001123
    a                      